In [1]:
import pandas as pd
import numpy as np
import duckdb
from fila_estacoes_utils import P_1 as P
from tqdm.notebook import tqdm

ouro_conn = duckdb.connect('ouro_db')
modelling_conn = duckdb.connect('modelling_db')

# ABT Estações

In [3]:
abt_estacoes_df = ouro_conn.execute(
"""
SELECT DISTINCT
    precipitacao.id_estacao
    ,dim_estacoes.latitude
    ,dim_estacoes.longitude
    ,dim_estacoes.vl_declividade
    ,dim_estacoes.vl_altitude
    ,dim_estacoes.vl_distancia_oceano
    ,dim_estacoes.vl_aspecto_relevo
    ,precipitacao.dt_medicao
    ,precipitacao.vl_precipitacao
    ,temperatura_maxima.vl_temperatura_maxima
    ,temperatura_media.vl_temperatura_media
    ,temperatura_minima.vl_temperatura_minima
    ,umidade_maxima.vl_umidade_relativa_maxima
    ,umidade_media.vl_umidade_relativa_media
    ,umidade_minima.vl_umidade_relativa_minima
    ,vento_2m_max.vl_velocidade_vento_2m_maxima
    ,vento_2m_media.vl_velocidade_vento_2m_media
    ,vento_10m_media.vl_velocidade_vento_10m_media
FROM
    fato_estacoes_precipitacao AS precipitacao
LEFT JOIN
    fato_estacoes_temperatura_maxima AS temperatura_maxima
    ON temperatura_maxima.id_estacao = precipitacao.id_estacao
    AND temperatura_maxima.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_temperatura_media AS temperatura_media
    ON temperatura_media.id_estacao = precipitacao.id_estacao
    AND temperatura_media.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_temperatura_minima AS temperatura_minima
    ON temperatura_minima.id_estacao = precipitacao.id_estacao
    AND temperatura_minima.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_umidade_relativa_maxima AS umidade_maxima
    ON umidade_maxima.id_estacao = precipitacao.id_estacao
    AND umidade_maxima.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_umidade_relativa_media AS umidade_media
    ON umidade_media.id_estacao = precipitacao.id_estacao
    AND umidade_media.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_umidade_relativa_minima AS umidade_minima
    ON umidade_minima.id_estacao = precipitacao.id_estacao
    AND umidade_minima.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_velocidade_vento_2m_maxima AS vento_2m_max
    ON vento_2m_max.id_estacao = precipitacao.id_estacao
    AND vento_2m_max.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_velocidade_vento_2m_media AS vento_2m_media
    ON vento_2m_media.id_estacao = precipitacao.id_estacao
    AND vento_2m_media.dt_medicao = precipitacao.dt_medicao
LEFT JOIN
    fato_estacoes_velocidade_vento_10m_media AS vento_10m_media
    ON vento_10m_media.id_estacao = precipitacao.id_estacao
    AND vento_10m_media.dt_medicao = precipitacao.dt_medicao
JOIN dim_estacoes
    ON precipitacao.id_estacao = dim_estacoes.id_estacao
""").fetch_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
modelling_conn.execute("""
CREATE OR REPLACE TABLE abt_estacoes AS
SELECT * FROM abt_estacoes_df
""")

ouro_conn.execute("""
CREATE OR REPLACE TABLE abt_estacoes AS
SELECT * FROM abt_estacoes_df
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

# ABT Produtos

In [2]:
abt_produtos_df = ouro_conn.execute(
"""
SELECT DISTINCT
   abt_estacoes.*
    ,chirps.vl_precipitacao AS vl_precipitacao_chirps
    ,cpc.vl_precipitacao AS vl_precipitacao_cpc
    ,cpc.vl_temperatura_maxima AS vl_temperatura_maxima_cpc
    ,cpc.vl_temperatura_minima AS vl_temperatura_minima_cpc
    ,gpm_final_run.vl_precipitacao AS vl_precipitacao_gpm_final_run
    ,gpm_late_run.vl_precipitacao AS vl_precipitacao_gpm_late_run
    ,power.vl_precipitacao AS vl_precipitacao_power
    ,power.vl_temperatura_maxima_2m_K AS vl_temperatura_maxima_2m_K_power
    ,power.vl_temperatura_media_2m_K AS vl_temperatura_media_2m_K_power
    ,power.vl_temperatura_minima_2m_K AS vl_temperatura_minima_2m_K_power
    ,power.vl_umidade_relativa_2m AS vl_umidade_relativa_2m_power
    ,power.vl_pressao_nivel_superficie AS vl_pressao_nivel_superficie_power
    ,power.vl_irradiancia_allsky AS vl_irradiancia_allsky_power
    ,power.vl_direcao_vento_10m AS vl_direcao_vento_10m_power
    ,power.vl_direcao_vento_2m AS vl_direcao_vento_2m_power
    ,power.vl_temperatura_orvalho_2m_K AS vl_temperatura_orvalho_2m_K_power
    ,power.vl_vento_10m AS vl_vento_10m_power
    ,power.vl_vento_medio_2m AS vl_vento_medio_2m_power
    ,power.vl_vento_maximo_2m AS vl_vento_maximo_2m_power
    ,power.vl_vento_maximo_10m AS vl_vento_maximo_10m_power
FROM abt_estacoes
JOIN fato_estacoes_latlon_produtos_df AS latlon
    ON abt_estacoes.id_estacao = latlon.id_estacao
LEFT JOIN fato_produto_chirps AS chirps
    ON latlon.lat_chirps = chirps.lat
    AND latlon.lon_chirps = chirps.lon
    AND abt_estacoes.dt_medicao = chirps.dt_medicao
LEFT JOIN fato_produto_cpc AS cpc
    ON latlon.lat_cpc = cpc.lat
    AND latlon.lon_cpc = cpc.lon
    AND abt_estacoes.dt_medicao = cpc.dt_medicao
LEFT JOIN fato_produto_gpm_final_run AS gpm_final_run
    ON latlon.lat_gpm_final_run = gpm_final_run.lat
    AND latlon.lon_gpm_final_run = gpm_final_run.lon
    AND abt_estacoes.dt_medicao = gpm_final_run.dt_medicao
LEFT JOIN fato_produto_gpm_late_run AS gpm_late_run
    ON latlon.lat_gpm_late_run = gpm_late_run.lat
    AND latlon.lon_gpm_late_run = gpm_late_run.lon
    AND abt_estacoes.dt_medicao = gpm_late_run.dt_medicao
LEFT JOIN fato_produto_power AS power
    ON latlon.lat_power = power.lat
    AND latlon.lon_power = power.lon
    AND abt_estacoes.dt_medicao = power.dt_medicao
""").fetch_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
modelling_conn.execute("""
CREATE OR REPLACE TABLE abt_produtos AS
SELECT * FROM abt_produtos_df
""")

ouro_conn.execute("""
CREATE OR REPLACE TABLE abt_produtos AS
SELECT * FROM abt_produtos_df
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

# Fila de estações de Prioridade

In [24]:
D = 30
α = 3
n_casas_decimais = 4

from fila_estacoes_utils import P_1,apply_P_as_column

fato_estacoes_base_fila_prioridade = ouro_conn.execute("SELECT * FROM fato_estacoes_base_fila_prioridade").fetch_df()
fato_estacoes_base_fila_prioridade["P_1"] = apply_P_as_column(fato_estacoes_base_fila_prioridade,P_1,D,α).round(n_casas_decimais)
fato_estacoes_base_fila_prioridade['P_1_Rank'] = fato_estacoes_base_fila_prioridade.groupby('id_estacao_base')['P_1'].rank(ascending=False,method='first').astype(int)


fato_estacoes_base_fila_prioridade = fato_estacoes_base_fila_prioridade.sort_values(by=['id_estacao_base','P_1_Rank']) \
    .reset_index(drop=True)

modelling_conn.execute("""
CREATE OR REPLACE TABLE fato_estacoes_fila_P_1 AS
SELECT * FROM fato_estacoes_base_fila_prioridade
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [27]:
fato_estacoes_base_fila_prioridade

,id_estacao_base,id_estacao_candidata,correlacao,pct_intersecao_precipitacao,vl_distancia_km,P_1,P_1_Rank
0,1,31029,0.563254,100.000000,3.00,0.5627,1
1,1,91,0.593248,100.000000,16.88,0.5035,2
2,1,927763,0.449028,95.255042,8.38,0.4186,3
3,1,927781,0.490810,100.000000,17.93,0.4045,4
4,1,927782,0.505842,100.000000,19.15,0.4014,5
...,...,...,...,...,...,...,...
19842565,1733225,926769,0.000000,0.000000,248.78,0.0000,4450
19842566,1733225,926162,0.000000,0.000000,547.39,0.0000,4451
19842567,1733225,926372,0.000000,0.000000,777.08,0.0000,4452
19842568,1733225,927897,0.000000,0.000000,492.20,0.0000,4453


# ABT Estações Vizinhas

In [3]:
def compact_row(row,cols,n_vizinhas):
    valores = row[cols].values
    valores_validos = [v for v in valores if pd.notnull(v)]
    valores_compactados = valores_validos + [np.nan] * (n_vizinhas - len(valores_validos))
    return pd.Series(valores_compactados, index=cols)

def compactar_vizinhas(df, n_vizinhas):
    colunas_modelo = [
        'vl_correlacao_estacao_vizinha_{}',
        'pct_intersecao_precipitacao_vizinha_{}',
        'vl_distancia_km_vizinha_{}',
        'vl_prioridade_vizinha_{}',
        'vl_precipitacao_vizinha_{}'
    ]
    
    for col_base in colunas_modelo:
        cols = [col_base.format(i) for i in range(1, n_vizinhas + 1)]
        matriz = df[cols].to_numpy()
        mask = ~np.isnan(matriz)
        matriz_compactada = np.full_like(matriz, np.nan)

        for i in tqdm(range(len(matriz)), desc=f"Compactando {col_base}"):
            valores_validos = matriz[i][mask[i]]
            matriz_compactada[i, :len(valores_validos)] = valores_validos

        df[cols] = matriz_compactada

    return df

def gerar_abt_estacoes_vizinhas(n_vizinhas=3, modelling_conn=modelling_conn):
    df = modelling_conn.execute(
    """
    SELECT
        abt_produtos.*
        ,vizinha_1.correlacao AS vl_correlacao_estacao_vizinha_1
        ,vizinha_1.pct_intersecao_precipitacao AS pct_intersecao_precipitacao_vizinha_1
        ,vizinha_1.vl_distancia_km AS vl_distancia_km_vizinha_1
        ,vizinha_1.P_1 AS vl_prioridade_vizinha_1
        ,abt_vizinha_1.vl_precipitacao AS vl_precipitacao_vizinha_1

        ,vizinha_2.correlacao AS vl_correlacao_estacao_vizinha_2
        ,vizinha_2.pct_intersecao_precipitacao AS pct_intersecao_precipitacao_vizinha_2
        ,vizinha_2.vl_distancia_km AS vl_distancia_km_vizinha_2
        ,vizinha_2.P_1 AS vl_prioridade_vizinha_2
        ,abt_vizinha_2.vl_precipitacao AS vl_precipitacao_vizinha_2

        ,vizinha_3.correlacao AS vl_correlacao_estacao_vizinha_3
        ,vizinha_3.pct_intersecao_precipitacao AS pct_intersecao_precipitacao_vizinha_3
        ,vizinha_3.vl_distancia_km AS vl_distancia_km_vizinha_3
        ,vizinha_3.P_1 AS vl_prioridade_vizinha_3
        ,abt_vizinha_3.vl_precipitacao AS vl_precipitacao_vizinha_3

        ,vizinha_4.correlacao AS vl_correlacao_estacao_vizinha_4
        ,vizinha_4.pct_intersecao_precipitacao AS pct_intersecao_precipitacao_vizinha_4
        ,vizinha_4.vl_distancia_km AS vl_distancia_km_vizinha_4
        ,vizinha_4.P_1 AS vl_prioridade_vizinha_4
        ,abt_vizinha_4.vl_precipitacao AS vl_precipitacao_vizinha_4

        ,vizinha_5.correlacao AS vl_correlacao_estacao_vizinha_5
        ,vizinha_5.pct_intersecao_precipitacao AS pct_intersecao_precipitacao_vizinha_5
        ,vizinha_5.vl_distancia_km AS vl_distancia_km_vizinha_5
        ,vizinha_5.P_1 AS vl_prioridade_vizinha_5
        ,abt_vizinha_5.vl_precipitacao AS vl_precipitacao_vizinha_5

        ,vizinha_6.correlacao AS vl_correlacao_estacao_vizinha_6
        ,vizinha_6.pct_intersecao_precipitacao AS pct_intersecao_precipitacao_vizinha_6
        ,vizinha_6.vl_distancia_km AS vl_distancia_km_vizinha_6
        ,vizinha_6.P_1 AS vl_prioridade_vizinha_6
        ,abt_vizinha_6.vl_precipitacao AS vl_precipitacao_vizinha_6

        ,vizinha_7.correlacao AS vl_correlacao_estacao_vizinha_7
        ,vizinha_7.pct_intersecao_precipitacao AS pct_intersecao_precipitacao_vizinha_7
        ,vizinha_7.vl_distancia_km AS vl_distancia_km_vizinha_7
        ,vizinha_7.P_1 AS vl_prioridade_vizinha_7
        ,abt_vizinha_7.vl_precipitacao AS vl_precipitacao_vizinha_7

        ,vizinha_8.correlacao AS vl_correlacao_estacao_vizinha_8
        ,vizinha_8.pct_intersecao_precipitacao AS pct_intersecao_precipitacao_vizinha_8
        ,vizinha_8.vl_distancia_km AS vl_distancia_km_vizinha_8
        ,vizinha_8.P_1 AS vl_prioridade_vizinha_8
        ,abt_vizinha_8.vl_precipitacao AS vl_precipitacao_vizinha_8

        ,vizinha_9.correlacao AS vl_correlacao_estacao_vizinha_9
        ,vizinha_9.pct_intersecao_precipitacao AS pct_intersecao_precipitacao_vizinha_9
        ,vizinha_9.vl_distancia_km AS vl_distancia_km_vizinha_9
        ,vizinha_9.P_1 AS vl_prioridade_vizinha_9
        ,abt_vizinha_9.vl_precipitacao AS vl_precipitacao_vizinha_9

        ,vizinha_10.correlacao AS vl_correlacao_estacao_vizinha_10
        ,vizinha_10.pct_intersecao_precipitacao AS pct_intersecao_precipitacao_vizinha_10
        ,vizinha_10.vl_distancia_km AS vl_distancia_km_vizinha_10
        ,vizinha_10.P_1 AS vl_prioridade_vizinha_10
        ,abt_vizinha_10.vl_precipitacao AS vl_precipitacao_vizinha_10

    FROM abt_produtos

    JOIN fato_estacoes_fila_P_1 AS vizinha_1
        ON abt_produtos.id_estacao = vizinha_1.id_estacao_base
        AND vizinha_1.P_1_Rank = 1
    LEFT JOIN abt_produtos AS abt_vizinha_1
        ON vizinha_1.id_estacao_candidata = abt_vizinha_1.id_estacao
        AND abt_produtos.dt_medicao = abt_vizinha_1.dt_medicao

    JOIN fato_estacoes_fila_P_1 AS vizinha_2
        ON abt_produtos.id_estacao = vizinha_2.id_estacao_base
        AND vizinha_2.P_1_Rank = 2
    LEFT JOIN abt_produtos AS abt_vizinha_2
        ON vizinha_2.id_estacao_candidata = abt_vizinha_2.id_estacao
        AND abt_produtos.dt_medicao = abt_vizinha_2.dt_medicao

    JOIN fato_estacoes_fila_P_1 AS vizinha_3
        ON abt_produtos.id_estacao = vizinha_3.id_estacao_base
        AND vizinha_3.P_1_Rank = 3
    LEFT JOIN abt_produtos AS abt_vizinha_3
        ON vizinha_3.id_estacao_candidata = abt_vizinha_3.id_estacao
        AND abt_produtos.dt_medicao = abt_vizinha_3.dt_medicao

    JOIN fato_estacoes_fila_P_1 AS vizinha_4
        ON abt_produtos.id_estacao = vizinha_4.id_estacao_base
        AND vizinha_4.P_1_Rank = 4
    LEFT JOIN abt_produtos AS abt_vizinha_4
        ON vizinha_4.id_estacao_candidata = abt_vizinha_4.id_estacao
        AND abt_produtos.dt_medicao = abt_vizinha_4.dt_medicao

    JOIN fato_estacoes_fila_P_1 AS vizinha_5
        ON abt_produtos.id_estacao = vizinha_5.id_estacao_base
        AND vizinha_5.P_1_Rank = 5
    LEFT JOIN abt_produtos AS abt_vizinha_5
        ON vizinha_5.id_estacao_candidata = abt_vizinha_5.id_estacao
        AND abt_produtos.dt_medicao = abt_vizinha_5.dt_medicao

    JOIN fato_estacoes_fila_P_1 AS vizinha_6
        ON abt_produtos.id_estacao = vizinha_6.id_estacao_base
        AND vizinha_6.P_1_Rank = 6
    LEFT JOIN abt_produtos AS abt_vizinha_6
        ON vizinha_6.id_estacao_candidata = abt_vizinha_6.id_estacao
        AND abt_produtos.dt_medicao = abt_vizinha_6.dt_medicao

    JOIN fato_estacoes_fila_P_1 AS vizinha_7
        ON abt_produtos.id_estacao = vizinha_7.id_estacao_base
        AND vizinha_7.P_1_Rank = 7
    LEFT JOIN abt_produtos AS abt_vizinha_7
        ON vizinha_7.id_estacao_candidata = abt_vizinha_7.id_estacao
        AND abt_produtos.dt_medicao = abt_vizinha_7.dt_medicao

    JOIN fato_estacoes_fila_P_1 AS vizinha_8
        ON abt_produtos.id_estacao = vizinha_8.id_estacao_base
        AND vizinha_8.P_1_Rank = 8
    LEFT JOIN abt_produtos AS abt_vizinha_8
        ON vizinha_8.id_estacao_candidata = abt_vizinha_8.id_estacao
        AND abt_produtos.dt_medicao = abt_vizinha_8.dt_medicao

    JOIN fato_estacoes_fila_P_1 AS vizinha_9
        ON abt_produtos.id_estacao = vizinha_9.id_estacao_base
        AND vizinha_9.P_1_Rank = 9
    LEFT JOIN abt_produtos AS abt_vizinha_9
        ON vizinha_9.id_estacao_candidata = abt_vizinha_9.id_estacao
        AND abt_produtos.dt_medicao = abt_vizinha_9.dt_medicao

    JOIN fato_estacoes_fila_P_1 AS vizinha_10
        ON abt_produtos.id_estacao = vizinha_10.id_estacao_base
        AND vizinha_10.P_1_Rank = 10
    LEFT JOIN abt_produtos AS abt_vizinha_10
        ON vizinha_10.id_estacao_candidata = abt_vizinha_10.id_estacao
        AND abt_produtos.dt_medicao = abt_vizinha_10.dt_medicao
    """).fetch_df()

    compacted_df = compactar_vizinhas(df, n_vizinhas=n_vizinhas)

    return compacted_df
    


modelling_conn.execute("SET memory_limit = '16GB'")


In [4]:
abt_estacoes_vizinhas_current_df = gerar_abt_estacoes_vizinhas(n_vizinhas=3)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Compactando vl_correlacao_estacao_vizinha_{}:   0%|          | 0/18194966 [00:00<?, ?it/s]

Compactando pct_intersecao_precipitacao_vizinha_{}:   0%|          | 0/18194966 [00:00<?, ?it/s]

Compactando vl_distancia_km_vizinha_{}:   0%|          | 0/18194966 [00:00<?, ?it/s]

Compactando vl_prioridade_vizinha_{}:   0%|          | 0/18194966 [00:00<?, ?it/s]

Compactando vl_precipitacao_vizinha_{}:   0%|          | 0/18194966 [00:00<?, ?it/s]

In [17]:
modelling_conn.execute("""
CREATE OR REPLACE TABLE abt_estacoes_3_vizinhas AS
SELECT * FROM abt_estacoes_vizinhas_current_df
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

# TESTING

In [ ]:
abt_estacoes_vizinhas = modelling_conn.execute("""SELECT * FROM abt_estacoes_vizinhas""").fetch_df()

# Train-Test split 0.7
# Particionar no tempo e na estação
# Em cada estação, treinar nos primeiros 70% dos dados e validar nos outros 30%

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
abt_estacoes_vizinhas

,id_estacao,latitude,longitude,vl_declividade,vl_altitude,vl_distancia_oceano,vl_aspecto_relevo,dt_medicao,vl_precipitacao,vl_temperatura_maxima,...,vl_vento_10m_power,vl_vento_medio_2m_power,vl_vento_maximo_2m_power,vl_vento_maximo_10m_power,vl_precipitacao_vizinha,vl_correlacao_vizinha,pct_intersecao_precipitacao_vizinha,vl_distancia_km_vizinha,estacao_vizinha_escolhida,vl_prioridade_vizinha
0,923901,-21.433300,-47.333302,1.74,595.0,287.119995,247.830002,2005-11-08,0.00000,NaN,...,2.915625,1.693750,2.654687,4.646875,0.000000,0.578020,97.120949,7.50,1.0,0.5527
1,921373,-19.415800,-41.729698,12.38,420.0,165.789993,299.420013,2011-01-20,0.00000,NaN,...,2.587500,1.656250,2.660938,3.617188,0.000000,0.588478,93.812805,24.77,1.0,0.3532
2,922797,-20.850000,-48.849998,3.47,534.0,419.540009,3.950000,2000-11-26,0.00000,NaN,...,1.726562,1.028125,2.393750,3.289062,0.000000,0.817498,100.000000,5.56,1.0,0.8123
3,922073,-20.296400,-40.933899,8.77,815.0,57.840000,274.839996,2016-07-23,0.00000,NaN,...,4.098437,3.068750,3.995312,5.370313,0.000000,0.657976,98.972565,15.33,1.0,0.5746
4,926460,-22.898100,-51.888100,1.03,498.0,421.429993,100.010002,2005-09-12,27.40000,NaN,...,3.442188,2.228125,3.157813,4.845313,36.200001,0.597978,98.415993,13.40,1.0,0.5404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18194961,924569,-22.106100,-41.471100,0.30,10.0,11.010000,168.690002,2001-05-09,0.00000,NaN,...,4.596875,4.067187,6.085938,7.054688,0.000000,0.278218,100.000000,29.91,1.0,0.1397
18194962,923682,-21.420300,-46.962799,1.25,581.0,277.839996,334.649994,2022-08-07,0.00000,NaN,...,2.546875,1.321875,2.435937,4.234375,0.000000,0.447699,96.758934,12.93,1.0,0.4011
18194963,577228,-23.093599,-46.971802,0.17,0.0,105.199997,30.959999,2018-07-16,0.00000,NaN,...,2.679688,1.490625,3.087500,4.393750,0.000000,0.755313,99.882149,5.02,1.0,0.7509
18194964,577228,-23.093599,-46.971802,0.17,0.0,105.199997,30.959999,2018-06-12,0.98577,NaN,...,4.554688,2.857812,4.981250,7.214063,4.138237,0.755313,99.882149,5.02,1.0,0.7509
